# Notebook for playing around, testing, speed comparisons 

In [1]:
from __future__ import division

import numpy as np
from gensim.models import Word2Vec 
import scipy.stats as stat
from scipy.special import gamma
import scipy
from numpy import pi
from numba import jit, vectorize, float32, float64
import time
import line_profiler
import nltk
import os
import cProfile
import IPython
ip = IPython.get_ipython()
ip.define_magic('lprun', line_profiler.magic_lprun)
%load_ext line_profiler
%load_ext autoreload


In [26]:
wordvecs = Word2Vec.load_word2vec_format(
    "/Users/michael/Documents/Gaussian_LDA-master/data/glove.wiki/glove.6B.50d.txt", binary=False)

In [ ]:
len(wordvecs.vocab.keys())

In [ ]:
cd GaussianLDA/

In [57]:
import FastGaussianLDA
%aimport FastGaussianLDA
%autoreload 1
# import GaussianLDA
# reload(Gauss_LDA)
# reload(GaussianLDA)

# corpus = ['apple banana mango melon',
#  'barrel cannon gun war',
#  'democrat republican senate politics']
corpus2 = ["apple orange mango melon ", "canvas art mural paint painting ", "pineapple kiwi grape strawberry ",
              "picture frame picasso sculpture art ", "coconut guava blueberry blackberry ", "statue monument art artist "]
# corpus2 = [sent * 5 for sent in corpus2]*4
wordvec_fileapth = "/Users/michael/Documents/Gaussian_LDA-master/data/glove.wiki/glove.6B.50d.txt"
start  = time.time()
g = FastGaussianLDA.Gauss_LDA(2, corpus2, word_vector_model=wordvecs)
g.fit(20)
print (time.time()-start)

Done processing corpus with 6 documents
There are 24 words that could be convereted to word vectors in your corpus 
There are 0 words that could NOT be converted to word vectors
Intialization complete
Starting fit
apple
orange
mango
melon
########################
 ###############
  ################# 0
canvas
art
mural
paint
painting
########################
 ###############
  ################# 1
pineapple
kiwi
grape
strawberry
########################
 ###############
  ################# 2
picture
frame
picasso
sculpture
art
########################
 ###############
  ################# 3
coconut
guava
blueberry
blackberry
########################
 ###############
  ################# 4
statue
monument
art
artist
########################
 ###############
  ################# 5
0 iterations complete
apple
orange
mango
melon
########################
 ###############
  ################# 0
canvas
art
mural
paint
painting
########################
 ###############
  ################# 1
pineappl

old time was 237 secs on 30 iterations without NUMBA

In [58]:
np.loadtxt("/Users/michael/Documents/GaussianLDA/output/iter10topic0Topic Mean.txt")

array([ 0.04701606,  0.06495298, -0.06701785,  0.12368204,  0.04580263,
        0.04126731, -0.15244943, -0.11145958, -0.00325656,  0.08293305,
       -0.03406654,  0.03001568,  0.16134068, -0.03776183,  0.04927907,
       -0.01430325, -0.08997902,  0.08577938,  0.039224  , -0.12702498,
        0.0261495 , -0.06010891,  0.04436867, -0.01811015,  0.00611593,
       -0.02520956, -0.09775412,  0.10198555,  0.05974979, -0.10871787,
        0.24333452, -0.09253282, -0.10494265,  0.10045677, -0.00074449,
        0.01816347, -0.1071244 ,  0.02481544,  0.04297808, -0.10480475,
        0.03284159,  0.00831267, -0.10501135, -0.10401834,  0.0992685 ,
        0.07110563, -0.01013705, -0.16764329,  0.00626053, -0.08539053])

In [50]:
np.savetxt("/Users/michael/Documents/GaussianLDA/output/test.txt", g.topic_params[0]["Topic Mean"])

In [ ]:
mean1 = g.topic_params[0]["Topic Mean"]; mean2 = g.topic_params[1]["Topic Mean"]; mean3 = g.topic_params[2]["Topic Mean"]
wordvecs.most_similar(positive=[mean1])

In [ ]:
wordvecs.most_similar(positive=[mean2])

In [ ]:
wordvecs.most_similar(positive=[mean3])

In [ ]:
newdoc = "fruit apple orange mango taco painting sculpture book art artist picasso"

In [ ]:
def myfunc(mat):
    np.linalg.det(mat)
    return np.linalg.inv(mat)
    
%prun myfunc(np.arange(100).reshape(10,10))

### Expirmenting with Numba accelerations:
- Using numpy ufuncs

In [ ]:
import numpy as np

In [3]:
a = np.cov(np.arange(50.0**2, dtype=np.float32).reshape(50,50))
a = np.eye(50.)*4
# scipy.linalg.cholesky(a, lower=True, overwrite_a=True)
x = np.arange(50)

In [4]:
from numba import generated_jit, vectorize

In [5]:
b = np.arange(1000*1000).reshape(1000,1000)

In [56]:
@jit(cache=True)
def func(val):
    np.sum(val, axis=0)

In [57]:
%%timeit
func(b)

The slowest run took 63.91 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 991 µs per loop


In [58]:
%%timeit
def func(val):
    np.sum(val, axis=0)

10000000 loops, best of 3: 90.2 ns per loop


- Using python operators

In [ ]:
@jit((float32[:,:]))
def chol_downdate(L, X):
        """
        Cholesky Rank 1 Update
        :param L: Lower triangle matrix from Cholesky Decomposition
        :param X: Word Vector with same column dimensionality as L
        :return: updated lower triangle matrix
        """
        # assert L.shape[1] == X.shape[0]
        # choldowndate(L.T, X)
        L_c = np.copy(L) # in-place computations are faster
        for k in range(X.shape[0]):
            r = np.sqrt(L_c[k, k]**2 - X[k]**2)
            c = r / L_c[k, k]
            s = X[k] / L_c[k, k]
            L_c[k, k] = r

            for i in range(k+1, X.shape[0]):
                L_c[i, k] = (L_c[i, k] - (s * X[i])) / c
                X[i] = (c * X[i]) - (s * L_c[i, k])
        if np.isnan(r) or np.isinf(r):
            print "YOU GOT Nans or infs: learn to code better shmuck"
            return L # good reason for making copy - return if downdate becomes unstable
        return L_c

In [ ]:
%%timeit
chol_downdate(a, x)

- Using numpy ufuncs is SLOWER

In [ ]:
@jit
def chol_downdate(L, X):
        """
        Cholesky Rank 1 Update
        :param L: Lower triangle matrix from Cholesky Decomposition
        :param X: Word Vector with same column dimensionality as L
        :return: updated lower triangle matrix
        """
        assert L.shape[1] == X.shape[0]
        # choldowndate(L.T, X)
        L_c = np.copy(L) # in-place computations are faster
        for k in range(X.shape[0]):
            r = np.sqrt(np.power(L_c[k, k],2) - np.power(X[k], 2))
            c = r / L_c[k, k]
            s = X[k] / L_c[k, k]
            L_c[k, k] = r

            for i in range(k+1, X.shape[0]):
                L_c[i, k] = (L_c[i, k] - (s * X[i])) / c
                X[i] = (c * X[i]) - (s * L_c[i, k])
        if np.isnan(r) or np.isinf(r):
            print "YOU GOT Nans or infs: learn to code better shmuck"
            return L # good reason for making copy - return if downdate becomes unstable
        return L_c

In [ ]:
%%timeit
chol_downdate(a, x)

In [9]:
np.sum(b, axis=0)

array([499500000, 499501000, 499502000, 499503000, 499504000, 499505000,
       499506000, 499507000, 499508000, 499509000, 499510000, 499511000,
       499512000, 499513000, 499514000, 499515000, 499516000, 499517000,
       499518000, 499519000, 499520000, 499521000, 499522000, 499523000,
       499524000, 499525000, 499526000, 499527000, 499528000, 499529000,
       499530000, 499531000, 499532000, 499533000, 499534000, 499535000,
       499536000, 499537000, 499538000, 499539000, 499540000, 499541000,
       499542000, 499543000, 499544000, 499545000, 499546000, 499547000,
       499548000, 499549000, 499550000, 499551000, 499552000, 499553000,
       499554000, 499555000, 499556000, 499557000, 499558000, 499559000,
       499560000, 499561000, 499562000, 499563000, 499564000, 499565000,
       499566000, 499567000, 499568000, 499569000, 499570000, 499571000,
       499572000, 499573000, 499574000, 499575000, 499576000, 499577000,
       499578000, 499579000, 499580000, 499581000, 

In [14]:
np.einsum('ik', b)

array([[     0,      1,      2, ...,    997,    998,    999],
       [  1000,   1001,   1002, ...,   1997,   1998,   1999],
       [  2000,   2001,   2002, ...,   2997,   2998,   2999],
       ..., 
       [997000, 997001, 997002, ..., 997997, 997998, 997999],
       [998000, 998001, 998002, ..., 998997, 998998, 998999],
       [999000, 999001, 999002, ..., 999997, 999998, 999999]])

In [ ]:
cd ../positive_text

In [27]:
stopwords = set(nltk.corpus.stopwords.words(fileids='english'))

In [28]:
directory = "/Users/michael/Documents/positive_text/"
docs = []
vocab = set(wordvecs.vocab.keys())
for path in os.listdir(directory)[1:]:
    for subpath in os.listdir(directory + path):
        try:
            with open(directory + path + '/' + subpath, 'r') as f:
                txt = f.read()
                txt = txt.split()                   
                txt = [word for word in txt if word in vocab and word.isalpha() 
                      and word not in stopwords and word != "ve"]
                txt = ' '.join(txt)
                if len(txt) > 5: docs.append(txt)
                f.close()
        except IOError: 
            for subsubpath in os.listdir(directory + path + '/' + subpath):
                try:
                    with open(directory + path + '/' + subpath + '/' + subsubpath, 'r') as f:
                        txt = f.read()
                        txt = txt.split()
                        txt = [word for word in txt if word in vocab and word.isalpha() 
                      and word not in stopwords and word != "ve"]
                        txt = ' '.join(txt)
                        docs.append(txt)
                except IOError: 
                    continue

In [ ]:
import numpy as np

In [ ]:
!touch cleandocs.txt

In [ ]:
pwd/

In [29]:
f = '/Users/michael/Documents/GaussianLDA/cleandocs.txt'
# np.savetxt(f, docs)
with open(f, 'w') as fi:
    for doc in docs:
        fi.write("%s\n" % doc)




In [30]:
f = '/Users/michael/Documents/GaussianLDA/cleandocs.txt'
with open(f, 'r') as fi:
    text = fi.read().splitlines()
    fi.close()

In [33]:
len(text)

5000

In [ ]:
len([doc.split() for doc in docs])

In [ ]:
docs[9]

In [ ]:
ls

In [ ]:
cd Insurance/

In [ ]:
with open("+PHI5_Ins-+_113529_SCAN0145.txt", 'r') as f:
    txt = f.read()
f.close()

In [ ]:
txt

In [ ]:
len(txt)

In [ ]:
nltk.tokenize.word_tokenize(txt)

In [41]:
np.where(a==4)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]))

In [2]:
np.einsum?